In [4]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
from Load import json_to_df
import numpy as np
import tensorflow.compat.v1.keras as keras 
from tensorflow.compat.v1.keras import layers
from tensorflow.compat.v1.keras.layers import Input, Lambda, Dense
from tensorflow.compat.v1.keras.models import Model, load_model
import tensorflow.compat.v1.keras.backend as K

In [7]:
class ElmoEmbeddingLayer(layers.Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable, name="{}_module".format(self.name))
        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)
        
    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result
    
    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [8]:
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = ElmoEmbeddingLayer()(input_text)
dense = layers.Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001))(embedding)
pred = layers.Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[input_text], outputs=pred)

AttributeError: 'ElmoEmbeddingLayer' object has no attribute 'trainable_weights_v1'

In [ ]:
embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [ ]:
load_model("elmo_model.hdf5", custom_objects=[ElmoEmbeddingLayer])

In [ ]:
model = None
model = build_model()
model.load_weights("model_org_.04-0.93.hdf5")